In [2]:
# Загрузка библиотек
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
from sklearn import datasets # для импорта данных
import seaborn as sns # библиотека для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графиков

%matplotlib inline


In [3]:
# загружаем датасет
boston = datasets.load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF["PRICE"] = boston.target
bostonDF.head()

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [ ]:
# CRIM: Per capita crime rate by town
# ZN: Proportion of residential land zoned for lots over 25,000 sq. ft
# INDUS: Proportion of non-retail business acres per town
# CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
# NOX: Nitric oxide concentration (parts per 10 million)
# RM: Average number of rooms per dwelling
# AGE: Proportion of owner-occupied units built prior to 1940
# DIS: Weighted distances to five Boston employment centers
# RAD: Index of accessibility to radial highways
# TAX: Full-value property tax rate per $10,000
# PTRATIO: Pupil-teacher ratio by town
# B: 1000(Bk — 0.63)², where Bk is the proportion of [people of African American descent] by town
# LSTAT: Percentage of lower status of the population
# MEDV: Median value of owner-occupied homes in $1000s

In [ ]:
# Хотим узнать, как обращаться к столбцам bostonDF
bostonDF.columns

In [ ]:
# полная матрица корреляций
# используем метод Pandas corr()
C = bostonDF.corr(method="pearson")
C

In [ ]:
# представим корреляционную матрицу в виде "тепловой карты" с помощью функции heatmap из библиотеки seaborn
plt.figure(figsize=(16, 6))  # размер графика
sns.heatmap(data=C, annot=True)

Строим регрессию из видео

In [ ]:
Data = bostonDF[["CRIM", "RM"]]
Data.head()

In [ ]:
np.shape(Data)

In [ ]:
# Создаем вектор из единиц для коэффициента w_0 и записываем все векторы в СТОЛБЦЫ матрицы признаков А
CRIM = Data["CRIM"]
RM = Data["RM"]
A = np.column_stack((np.ones(506), CRIM, RM))
A

In [ ]:
# Добавим настройку для удобного чтения значений А
np.set_printoptions(suppress=True)
A

In [ ]:
# Создаем целевой вектор
y = bostonDF[["PRICE"]]  # объект типа dataframe - то, что нужно
y_s = bostonDF["PRICE"]  #  объект типа series - не подойдет

In [ ]:
type(y)

In [ ]:
type(y_s)

In [ ]:
# вычислим OLS оценку для коэффициентов
w_hat = np.linalg.inv(A.T @ A) @ A.T @ y
w_hat

In [ ]:
# прогноз
# добавились данные по новому городку:
CRIM_new = 0.1
RM_new = 8
# делаем прогноз типичной соимости дома
PRICE_new = w_hat.iloc[0] + w_hat.iloc[1] * CRIM_new + w_hat.iloc[2] * RM_new
PRICE_new

In [ ]:
# короткий способ сделать прогноз
new = np.array([1, CRIM_new, RM_new])
new @ w_hat

In [ ]:
# классическая OLS регрессия в numpy одной командой
np.linalg.lstsq(A, y, rcond=None)

Стандартизация данных

In [ ]:
Data.head()

In [ ]:
# метод .mean() позволяет вычислить арифметическое среднее значение вектора
meanCRIM = Data["CRIM"].mean()
meanRM = Data["RM"].mean()
mean_y = y.mean()
print("mean value of CRIME:", meanCRIM)
print("mean value of RM:", meanRM)
print("mean value of PRICE:", mean_y)

In [ ]:
# Центрирование
CRIM_c = Data["CRIM"] - meanCRIM
RM_c = Data["RM"] - meanRM
y_c = y - mean_y
print("CRIME до центрирования:\n", CRIM.head(4))
print("CRIME после центрирования:\n", CRIM_c.head(4))

In [ ]:
print("среднее арифметическое вектора CRIM после центрирования:", CRIM_c.mean())

In [ ]:
# вычисляем длины векторов для нормирования
CRIM_c_norm = np.linalg.norm(CRIM_c)
RM_c_norm = np.linalg.norm(RM_c)
y_c_norm = np.linalg.norm(y_c)
print("norm of CRIME:\n", CRIM_c_norm)

In [ ]:
# Нормирование: делим каждый центрированный вектор на его длину
CRIM_st = CRIM_c / CRIM_c_norm
RM_st = RM_c / RM_c_norm
y_st = y_c / y_c_norm
print("CRIME до центрирования:\n", CRIM.head(4))
print("CRIME после центрирования:\n", CRIM_c.head(4))
print("CRIME после нормирования:\n", CRIM_st.head(4))

In [ ]:
# Матрица центрированных признаков - БЕЗ константы!
A_st = np.column_stack(
    (
        CRIM_st,
        RM_st,
    )
)
A_st

In [ ]:
# OLS оценка коэффициентов центрированной регрессии
w_hat_st = np.linalg.inv(A_st.T @ A_st) @ A_st.T @ y_st.values
w_hat_st

In [ ]:
# добавились данные по новому городку:
CRIM_new = 0.1
RM_new = 8
# чтобы сделать прогноз по новым данным, их тоже нужно стандартизировать

In [ ]:
# Стандартизация новых данных
CRIM_new_st = (CRIM_new - meanCRIM) / CRIM_c_norm
RM_new_st = (RM_new - meanRM) / RM_c_norm
print("new CRIME st:", CRIM_new_st)
print("new RM st:", RM_new_st)

In [ ]:
# Прогноз стандартизированного y
y_st_new = w_hat_st[0] * CRIM_new_st + w_hat_st[1] * RM_new_st
print("new PRICE st predict:", y_st_new)

Стандартизированный прогноз для нас может не иметь никакого смысла сам по себе, 

поэтому его необходимо пересчитать обратно.

Для этого сделаем операции, обратные стандартизации - умножим на длину центрированного вектора y и прибавим среднее


In [ ]:
# Пересчет стандартизированного прогноза в понятный
y_new = y_st_new * y_c_norm + mean_y
print("new PRICE predict:", y_new)

In [ ]:
# Пересчет стандартизированных коэффициентов в обычные
# здесь создаем вектор из единиц, который далее заполним нужными значениями
w_hat_not_st = np.ones((3, 1))

In [ ]:
# Пересчет стандартизированных  коэффициентов в обычные
w_hat_not_st[0] = (
    -w_hat_st[0] * meanCRIM / CRIM_c_norm - w_hat_st[1] * meanRM / RM_c_norm
) * y_c_norm + y.mean()
w_hat_not_st[1] = (w_hat_st[0] / CRIM_c_norm) * y_c_norm
w_hat_not_st[2] = (w_hat_st[1] / RM_c_norm) * y_c_norm
w_hat_not_st

In [ ]:
# Сравнение с ранее полученными обычными коэффициентами
w_hat

In [ ]:
# Матрица Грама стандартизированных признаков
A_st.T @ A_st

In [ ]:
# Матрица корреляций обычных признаков
Data.corr(method="pearson")

In [ ]:
# Стандартизированные признаки ортогональны вектору констант

In [ ]:
CRIM_st @ np.ones(506)

In [ ]:
RM_st @ np.ones(506)